# IMPORTING THE LIBRARIES

In [2]:
pip install SpeechRecognition

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [3]:
import speech_recognition as sr

In [4]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
import time
import difflib
import numpy
import keras
import tensorflow
import json
import pickle
import random
import os

In [5]:
import sys
print(sys.version)

3.10.9 | packaged by Anaconda, Inc. | (main, Mar  1 2023, 18:18:15) [MSC v.1916 64 bit (AMD64)]


In [6]:
import tensorflow as tf
print(tf.__version__)

2.12.0


# PRE- PROCESSING THE DATA

In [7]:
with open("intents (1).json") as file:
    data = json.load(file)

try:
    with open("data.pickle", "rb") as f:
        words, labels, training, output = pickle.load(f)
except:
    words = []
    labels =[]
    docs_patt = []
    docs_tag = []

#TOKENISATION & STEMMING
    for intent in data["intents"]:
        for pattern in intent["patterns"]:
            wrds = nltk.word_tokenize(pattern)
            for item in wrds:
                words.extend(wrds)
                docs_patt.append(wrds)
                docs_tag.append(intent["tag"])
                if intent["tag"] not in labels:
                    labels.append(intent["tag"])
    words = [stemmer.stem(w.lower()) for w in words]
    words = sorted(list(set(words)))
    labels = sorted(labels)

    training = []
    output = []

    out_empty = [0 for _ in range(len(labels))]
# BAG OF WORDS - FEATURE ENGINEERING
    for x, doc in enumerate(docs_patt):
        bag = []
        wrds = [stemmer.stem(w.lower()) for w in doc]
        for w in words:
            if w in wrds:
                bag.append(1)
            else:
                bag.append(0)
        output_row = out_empty[:]
        output_row[labels.index(docs_tag[x])] = 1

        training.append(bag)
        output.append(output_row)

    training = numpy.array(training)
    output = numpy.array(output)

    with open("data.pickle", "wb") as f:
        pickle.dump((words, labels, training, output), f)

# MODEL BUILDING

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Create a Sequential model
model = Sequential()

# Add an input layer with the shape of your input data
model.add(Dense(units=8, input_shape=(len(training[0]),), activation='relu'))

# Add hidden layers
model.add(Dense(units=8, activation='relu'))
model.add(Dense(units=len(output[0]), activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
# Try loading the model, and if it doesn't exist, train and save it
try:
    model.load_weights("model.keras")
except:
    history = model.fit(training, output, epochs=100, batch_size=8, verbose=1)
    model.save_weights("model.keras")

# INPUT PRE PROCESSING

In [10]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1

    return numpy.array(bag)

In [11]:
def words_to_list(s):
    a = []
    ns = ""
    s = s + " " 
    for i in range(len(s)):
        if s[i] == " ":
            a.append(ns)
            ns = ""
        else:
            ns = ns + s[i]
    a = list(set(a))
    return a

In [12]:
# pass the file in this fuction to create a dictionary of unique vocabulary
def json_to_dictionary(data):
    dictionary = []
    fil_dict= []
    vocalubary = []
    for i in data["intents"]:
        for pattern in i["patterns"]:
            vocalubary.append(pattern.lower())
    for i in vocalubary:
        dictionary.append(words_to_list(i))
    for i in range(len(dictionary)):
        for word in dictionary[i]:
            fil_dict.append(word)
    return list(set(fil_dict))

# this fuction checks the spelling in the sentence
chatbot_vocabulary = json_to_dictionary(data)

In [13]:
def word_checker(s):
    correct_string = ""
    for word in s.casefold().split():
        if word not in chatbot_vocabulary:
            suggestion = difflib.get_close_matches(word, chatbot_vocabulary)
            for x in suggestion:
                pass
            if len(suggestion) == 0:
                pass
            else:
                correct_string = correct_string + " " + str(suggestion[0])
        else:
            correct_string = correct_string + " " + str(word)

    return correct_string 

# SPEECH RECOGNITION

In [10]:
##ignore
pip install speech_recognition

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement speech_recognition (from versions: none)
ERROR: No matching distribution found for speech_recognition


In [6]:
pip install pyttsx3

  Obtaining dependency information for comtypes from https://files.pythonhosted.org/packages/c2/a7/fe4bd49b5c4afa7a7ed3852abda6909e48c00715e6a134e47055381113aa/comtypes-1.2.0-py2.py3-none-any.whl.metadata
   ---------------------------------------- 0.0/184.3 kB ? eta -:--:--
   ---------------------------------------- 0.0/184.3 kB ? eta -:--:--
   ------ --------------------------------- 30.7/184.3 kB ? eta -:--:--
   -------- ------------------------------ 41.0/184.3 kB 495.5 kB/s eta 0:00:01
   -------- ------------------------------ 41.0/184.3 kB 495.5 kB/s eta 0:00:01
   ----------------- --------------------- 81.9/184.3 kB 416.7 kB/s eta 0:00:01
   ----------------- --------------------- 81.9/184.3 kB 416.7 kB/s eta 0:00:01
   ----------------- --------------------- 81.9/184.3 kB 416.7 kB/s eta 0:00:01
   ------------------- ------------------- 92.2/184.3 kB 291.5 kB/s eta 0:00:01
   ------------------- ------------------- 92.2/184.3 kB 291.5 kB/s eta 0:00:01
   ------------------

In [14]:
import speech_recognition as sr
r=sr.Recognizer()
import pyttsx3
engine = pyttsx3.init()

In [15]:
def bot_speaking(message):
    engine.say(message)
    engine.runAndWait()
    if engine._inLoop:
        engine.endLoop()

In [16]:
def get_input():
    with sr.Microphone() as source:
        #print("Say something!!!");
        bot_speaking("Hey mate say something")
        audio=r.listen(source,timeout=0)
        #print("Perfect, Thanks!")
        bot_speaking("Perfect, Thanks!")
    try:
        msg=r.recognize_google(audio)
        print("TEXT: "+msg); #r.recognize(audio,language='hi-IN')
        bot_speaking("you said "+msg)
        return msg
    except:
        #print("Dude it's not working :(")
        bot_speaking("Sorry mate! It's not working")
        pass;

# CHAT FUNCTION

In [17]:
def prepare_input(inp_x, words):
    # Create a bag of words representation for the input
    # You may need to modify this function to match your preprocessing logic
    input_data = bag_of_words(inp_x, words)

    # Convert the input data to a numpy array and reshape it to match the model's input shape
    input_data = np.array(input_data).reshape(1, -1)  # Reshape to (1, len(words))

    return input_data

In [18]:
import numpy as np

In [19]:
def chat():
    print("VNR :Hi! I am your personal bot. I am here to answer queries on VNRVJIET")
    while True:
        #inp=str(get_input())
        inp=input("YOU : ")
        if inp.lower() == "quit"or inp==None:
            break
        inp_x = word_checker(inp)
        input_data = prepare_input(inp_x, words)
        results = model.predict(input_data)[0]
        results_index = numpy.argmax(results)
        tag = labels[results_index]
        #print(results[results_index])
        if results[results_index] >= 0.9:
            for tg in data["intents"]:
                if tg['tag'] == tag:
                    responses = tg['responses']
                    ms= random.choice(responses)
                    print("VNR : "+ms)
                    bot_speaking(ms)
        else:
            print("VNR : Sorry, I don't know how to answer that yet ")
            bot_speaking("Sorry, I don't know how to answer that yet")
chat()

VNR :Hi! I am your personal bot. I am here to answer queries on VNRVJIET
YOU : tell me about canteen
1/1 [==============================] - 2s 2s/step
VNR : VNRVJIET has canteen with variety of food available ,types of stalls like 
 1)fast food 
2)bakery 
3)soft drinks 
4)Regular meals 
5)coffee shop 
6)cafeteria
YOU : tell me about hostel
1/1 [==============================] - 0s 30ms/step
VNR :  VNRVJIET has well-designed state-of-the-art hostels both for boys and girls.
It has lot of facilities from hot water to gym and sport facilities.
 Visit http://vnrvjiet.ac.in/hostel.php
YOU : tell about departments in vnrvjiet
1/1 [==============================] - 0s 24ms/step
VNR : A Student at vnr can be admitted in 3 ways 1)A-cat(eamcet,spot admissions) 2)B-cat(jee main) 3)Nri(12th marks).Last year cutoff ranks are provided for reference http://www.vnrvjiet.ac.in/download/admissions/FIRST_LAST_EAMCET_2019_RANKS.pdf
YOU : quit


In [16]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to C:\Users\Hari
[nltk_data]     Priya\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

# GUI APPLICATION - CHATBOT

In [20]:
from tkinter import *

BG_GRAY = "#ABB2B9"
BG_COLOR = "#17202A"
TEXT_COLOR = "#EAECEE"

FONT = "Helvetica 14"
FONT_BOLD = "Helvetica 13 bold"
bot_name ="VNR-BOT"
def get_response(msg):
    #print("Hi! I am your personal bot. I am here to answer queries on VNRVJIET")
    while True:
        inp=msg
        if inp.lower() == "quit"or inp==None:
            break
        inp_x = word_checker(inp)
        input_data = prepare_input(inp_x, words)
        results = model.predict(input_data)[0]
#         results = model.predict([bag_of_words(inp_x, words)])[0]
        results_index = numpy.argmax(results)
        tag = labels[results_index]
        #print(results[results_index])
        if results[results_index] >= 0.9:
            for tg in data["intents"]:
                if tg['tag'] == tag:
                    responses = tg['responses']
                    ms= random.choice(responses)
                    return ms
        else:
            return " Sorry, I don't know how to answer that yet "

class ChatApplication:
    
    def __init__(self):
        self.window = Tk()
        self._setup_main_window()
        
    def run(self):
        self.window.mainloop()
        
    def _setup_main_window(self):
        self.window.title("Chat")
        #self.window.resizable(width=False, height=False)
        self.window.configure(width=470, height=550, bg=BG_COLOR)
        
        # head label
        head_label = Label(self.window, bg=BG_COLOR, fg=TEXT_COLOR,
                           text="WELCOME TO VNRVJIET CHAT-BOT", font=FONT_BOLD, pady=10)
        head_label.place(relwidth=1)
        
        # tiny divider
        line = Label(self.window, width=450, bg=BG_GRAY)
        line.place(relwidth=1, rely=0.07, relheight=0.012)
        
        # text widget
        self.text_widget = Text(self.window, width=20, height=2, bg=BG_COLOR, fg=TEXT_COLOR,
                                font=FONT, padx=5, pady=5)
        self.text_widget.place(relheight=0.745, relwidth=1, rely=0.08)
        self.text_widget.configure(cursor="arrow", state=DISABLED)
        
        # scroll bar
        scrollbar = Scrollbar(self.text_widget)
        scrollbar.place(relheight=1, relx=0.974)
        scrollbar.configure(command=self.text_widget.yview)
        
        # bottom label
        bottom_label = Label(self.window, bg=BG_GRAY, height=80)
        bottom_label.place(relwidth=1, rely=0.825)
        
        # message entry box
        self.msg_entry = Entry(bottom_label, bg="#2C3E50", fg=TEXT_COLOR, font=FONT)
        self.msg_entry.place(relwidth=0.74, relheight=0.06, rely=0.008, relx=0.011)
        self.msg_entry.focus()
        self.msg_entry.bind("<Return>", self._on_enter_pressed)
        
        # send button
        send_button = Button(bottom_label, text="Send", font=FONT_BOLD, width=20, bg=BG_GRAY, command=lambda: self._on_enter_pressed(None))
        send_button.place(relx=0.77, rely=0.008, relheight=0.06, relwidth=0.22)
     
    def _on_enter_pressed(self, event):
        msg = self.msg_entry.get()
        self._insert_message(msg, "You")
        
    def _insert_message(self, msg, sender):
        if not msg:
            return
        
        self.msg_entry.delete(0, END)
        msg1 = f"{sender}: {msg}\n\n"
        self.text_widget.configure(state=NORMAL)
        self.text_widget.insert(END, msg1)
        self.text_widget.configure(state=DISABLED)
        
        msg2 = f"{bot_name}: {get_response(msg)}\n\n"
        bot_speaking(get_response(msg))
        self.text_widget.configure(state=NORMAL)
        self.text_widget.insert(END, msg2)
        self.text_widget.configure(state=DISABLED)
        
        self.text_widget.see(END)
             
        
if __name__ == "__main__":
    app = ChatApplication()
    app.run()

1/1 [==============================] - 0s 22ms/step
